## Analisis de la base de datos de Chinook Database

![](https://cdn.sqlitetutorial.net/wp-content/uploads/2015/11/sqlite-sample-database-color.jpg)

URL dB: https://cdn.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip

In [2]:
import sqlite3
import requests
import zipfile
import os

In [4]:
PATH = ".\\datafile\\"
URL = "https://cdn.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip"

filename = URL.split("/")[-1]
r = requests.get(URL)

try:
    print(f"Descargando {filename} de {URL}")
    # Se guarda el archivo ZIP
    with open(filename, mode='wb') as file:
        file.write(r.content)
    
    # Se descompime en archivo ZIP en el directorio de PATH
    print(f"Descomprimiendo el archivo {filename}")
    with zipfile.ZipFile(filename, mode='r') as zip_file:
        zip_file.extractall(PATH)
    
    # Se elimina el archivo ZIP
    os.remove(filename)
    print("Hecho!")
    
except:
    print(f"No se puede descargar el archivo del {URL}")

Descargando chinook.zip de https://cdn.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip
Descomprimiendo el archivo chinook.zip
Hecho!


## Inspeccionar la base de datos

In [5]:
database_file = ".\\datafile\\chinook.db"
conn = sqlite3.connect(database_file)

In [6]:
# LISTADO DE TABLAS EN LA BASE DE DATOS
with conn:
    cur = conn.cursor()
    sql = "SELECT name FROM sqlite_master WHERE type = 'table'"
    cur.execute(sql)
    
    print(f"Listado de tablas en {database_file}:")
    print()
    for item in cur:
        print("   *", item[0])

Listado de tablas en .\datafile\chinook.db:

   * albums
   * sqlite_sequence
   * artists
   * customers
   * employees
   * genres
   * invoices
   * invoice_items
   * media_types
   * playlists
   * playlist_track
   * tracks
   * sqlite_stat1


In [7]:
# LISTADO DE LAS COLUMNAS DE UNA TABLA
tabla = 'invoices'

with conn:
    cur = conn.cursor()
    sql = f"SELECT * FROM {tabla}"
    cur.execute(sql)
    
    print(f"Listado de columnas en tabla '{tabla}':")
    print()
    for item in cur.description:
        print("  -", item[0])

Listado de columnas en tabla 'invoices':

  - InvoiceId
  - CustomerId
  - InvoiceDate
  - BillingAddress
  - BillingCity
  - BillingState
  - BillingCountry
  - BillingPostalCode
  - Total


## Aplicacion en tkinter para la inspeccion de una dB

In [12]:
import sqlite3
import tkinter as tk
import tkinter.ttk as tkk
from tkinter.filedialog import askopenfilename
from tkinter.messagebox import askokcancel, showinfo, showerror

class Database:
    def __init__(self, datafile):
        self.__conn = sqlite3.connect(datafile)
        self.__cur = self.__conn.cursor()
        
    def __del__(self):
        self.__conn.close()
    
    def lista_tablas(self):
        sql = "SELECT name FROM sqlite_master WHERE type = 'table'"
        self.__cur.execute(sql)
        return [item[0] for item in self.__cur]
    
    def lista_columnas(self, tabla):
        sql = f"SELECT * FROM {tabla}"
        self.__cur.execute(sql)
        return [item[0] for item in self.__cur.description]
   
 
class App:
    def __init__(self, master):
        self.master = master
        self.master.title("Inspeccion dB")
        self.master.protocol("WM_DELETE_WINDOW", self.close_app)
        
        # ------------------------ Menu ------------------------------
        self.menu = tk.Menu(self.master)
        self.master.config(menu=self.menu)
        
        self.menuArchivo = tk.Menu(self.menu, tearoff=False)
        self.menuAyuda = tk.Menu(self.menu, tearoff=False)
        
        self.menuArchivo.add_command(label="Abrir archivo", command=self.open_file)
        self.menuArchivo.add_separator()
        self.menuArchivo.add_command(label="Salir", command=self.close_app)
        
        self.menuAyuda.add_command(label="Acerca de...", command=self.show_info)
        
        self.menu.add_cascade(label="Archivo", menu=self.menuArchivo)
        self.menu.add_cascade(label="Ayuda", menu=self.menuAyuda)
        
        # --------------------------- UX ------------------------------
        frm = tk.Frame(self.master)
        frm.pack(padx=10, pady=10)
        
        frm1 = tk.LabelFrame(frm, text="Tablas", padx=10, pady=10)
        frm2 = tk.LabelFrame(frm, text="Columnas", padx=10, pady=10)
        
        frm1.pack(side=tk.LEFT, padx=10, pady=10)
        frm2.pack(side=tk.LEFT, padx=10, pady=10)
        
        self.scrYTablas = tk.Scrollbar(frm1, orient='vertical')
        self.lstTablas = tk.Listbox(frm1, height=12, width=30, yscrollcommand=self.scrYTablas.set)
        self.scrYTablas.config(command=self.lstTablas.yview)
        
        self.lstTablas.pack(side=tk.LEFT)
        self.scrYTablas.pack(side=tk.LEFT, expand=True, fill=tk.Y)
        
        self.scrYColumnas = tk.Scrollbar(frm2, orient='vertical')
        self.lstColumnas = tk.Listbox(frm2, height=12, width=30, yscrollcommand=self.scrYColumnas.set)
        self.scrYColumnas.config(command=self.lstColumnas.yview)
        
        self.lstColumnas.pack(side=tk.LEFT)
        self.scrYColumnas.pack(side=tk.LEFT, expand=True, fill=tk.Y)
        
        self.lstTablas.bind("<<ListboxSelect>>", self.show_columns)
        
        self.statusBar = tk.Label(self.master, text="", bd=1, relief=tk.SUNKEN, anchor=tk.W)
        self.statusBar.pack(side=tk.BOTTOM, fill=tk.X)
        
        self.lstTablas.bind("<Enter>", lambda x: self.statusBar.config(text="Informacion de las tablas de la dB"))
        self.lstTablas.bind("<Leave>", lambda x: self.statusBar.config(text=""))
        self.lstColumnas.bind("<Enter>", lambda x: self.statusBar.config(text="Informacion de las columnas de una tabla"))
        self.lstColumnas.bind("<Leave>", lambda x: self.statusBar.config(text=""))
        
        
    def open_file(self):
        filename = askopenfilename()
        
        if filename != '':
            if filename.endswith(".db"):
                self.dB = Database(filename)
                self.master.title(f"Inspeccion dB - {filename}")
                
                for item in self.dB.lista_tablas():
                    self.lstTablas.insert(tk.END, item)
                
            else:
                showerror("Error", "Solo acepta archivo sqlite3 dB")
              
    def show_columns(self, event):
        try:
            tabla = self.lstTablas.get(self.lstTablas.curselection())
            self.lstColumnas.delete(0, tk.END)

            for item in self.dB.lista_columnas(tabla):
                self.lstColumnas.insert(tk.END, item)
        except:
            pass
        
    def close_app(self):
        if askokcancel("Salir", "¿Desea salir de la aplicacion?"):
            self.master.destroy()
            
    def show_info(self):
        _ = showinfo("Acerca de...", "Inspeccion dB (Python - tkinter)\n\nUPC 2020")
            
        
root = tk.Tk()
app = App(root)
root.mainloop()

## Mineria de datos

In [5]:
# Esta Metallica en la lista de artistas?
with conn:
    cur = conn.cursor()
    
    sql = "SELECT * FROM artists WHERE name LIKE '%met%'"
    cur.execute(sql)
    
    for item in cur:
        print(item)

(47, 'Hermeto Pascoal')
(50, 'Metallica')


In [14]:
# Cuales son los albums de Metallica en la tienda?
with conn:
    cur = conn.cursor()
    
    sql = """SELECT title 
             FROM albums 
             JOIN artists
             ON albums.artistId = artists.artistId
             WHERE name = ? 
             ORDER BY title"""
    cur.execute(sql, ("Metallica",))
    
    for idx, item in enumerate(cur, start=1):
        print(f"[{idx:2}]: {item[0]}")
        
    sql = """SELECT COUNT(title )
             FROM albums 
             JOIN artists
             ON albums.artistId = artists.artistId
             WHERE name = ?"""
    
    print("\nNumero de albums:", cur.execute(sql, ("Metallica",)).fetchone()[0])    

[ 1]: ...And Justice For All
[ 2]: Black Album
[ 3]: Garage Inc. (Disc 1)
[ 4]: Garage Inc. (Disc 2)
[ 5]: Kill 'Em All
[ 6]: Load
[ 7]: Master Of Puppets
[ 8]: ReLoad
[ 9]: Ride The Lightning
[10]: St. Anger

Numero de albums: 10


In [20]:
# Listado de los tracks por albums de Metallica
with conn:
    cur = conn.cursor()
    
    sql = """SELECT albumId, title 
             FROM albums 
             JOIN artists
             ON albums.artistId = artists.artistId
             WHERE name = ? 
             ORDER BY title"""
    cur.execute(sql, ("Metallica",))
    
    albums = [item for item in cur]
    
    for id_, album in albums:
        print("\n", album)
        sql = "SELECT name FROM tracks WHERE albumId = ?"
        tracks = cur.execute(sql, (id_,)).fetchall()
        
        for track in tracks:
            print(f"   * {track[0]}")
    


 ...And Justice For All
   * Blackened
   * ...And Justice For All
   * Eye Of The Beholder
   * One
   * The Shortest Straw
   * Harvester Of Sorrow
   * The Frayed Ends Of Sanity
   * To Live Is To Die
   * Dyers Eve

 Black Album
   * Enter Sandman
   * Sad But True
   * Holier Than Thou
   * The Unforgiven
   * Wherever I May Roam
   * Don't Tread On Me
   * Through The Never
   * Nothing Else Matters
   * Of Wolf And Man
   * The God That Failed
   * My Friend Of Misery
   * The Struggle Within

 Garage Inc. (Disc 1)
   * Free Speech For The Dumb
   * It's Electric
   * Sabbra Cadabra
   * Turn The Page
   * Die Die My Darling
   * Loverman
   * Mercyful Fate
   * Astronomy
   * Whiskey In The Jar
   * Tuesday's Gone
   * The More I See

 Garage Inc. (Disc 2)
   * Helpless
   * The Small Hours
   * The Wait
   * Crash Course In Brain Surgery
   * Last Caress/Green Hell
   * Am I Evil?
   * Blitzkrieg
   * Breadfan
   * The Prince
   * Stone Cold Crazy
   * So What
   * Killing Ti

In [25]:
# Cuanto es el costo de los tracks de Metallica
with conn:
    cur = conn.cursor()
    
    sql = """SELECT tracks.name, tracks.unitPrice 
             FROM tracks
             JOIN albums
             JOIN artists 
             ON tracks.albumId = albums.albumId 
             AND albums.artistId = artists.artistId 
             WHERE artists.name = ?"""
    
    cur.execute(sql, ("Metallica",))
    
    for item in cur:
        print(item)

('Free Speech For The Dumb', 0.99)
("It's Electric", 0.99)
('Sabbra Cadabra', 0.99)
('Turn The Page', 0.99)
('Die Die My Darling', 0.99)
('Loverman', 0.99)
('Mercyful Fate', 0.99)
('Astronomy', 0.99)
('Whiskey In The Jar', 0.99)
("Tuesday's Gone", 0.99)
('The More I See', 0.99)
('Enter Sandman', 0.99)
('Sad But True', 0.99)
('Holier Than Thou', 0.99)
('The Unforgiven', 0.99)
('Wherever I May Roam', 0.99)
("Don't Tread On Me", 0.99)
('Through The Never', 0.99)
('Nothing Else Matters', 0.99)
('Of Wolf And Man', 0.99)
('The God That Failed', 0.99)
('My Friend Of Misery', 0.99)
('The Struggle Within', 0.99)
('Helpless', 0.99)
('The Small Hours', 0.99)
('The Wait', 0.99)
('Crash Course In Brain Surgery', 0.99)
('Last Caress/Green Hell', 0.99)
('Am I Evil?', 0.99)
('Blitzkrieg', 0.99)
('Breadfan', 0.99)
('The Prince', 0.99)
('Stone Cold Crazy', 0.99)
('So What', 0.99)
('Killing Time', 0.99)
('Overkill', 0.99)
('Damage Case', 0.99)
('Stone Dead Forever', 0.99)
('Too Late Too Late', 0.99)
('Hi

In [27]:
# Cual es la facturacion de Metallica en la tienda
with conn:
    cur = conn.cursor()
    
    sql = """SELECT SUM(invoice_items.unitPrice) 
             FROM invoice_items 
             JOIN tracks
             JOIN albums
             JOIN artists
             ON invoice_items.trackId = tracks.trackId 
             AND tracks.albumId = albums.albumId 
             AND albums.artistId = artists.artistId 
             WHERE artists.name = ?"""
    
    monto = cur.execute(sql, ("Metallica",)).fetchone()[0]
    
    print(f"Facturacion: {monto:.2f} USD")

Facturacion: 90.09 USD


In [36]:
# Los paises Top5 que consumen productos de Metallica
with conn:
    cur = conn.cursor()
    
    sql = """SELECT SUM(invoice_items.unitPrice), customers.country 
             FROM customers
             JOIN invoices
             JOIN invoice_items 
             JOIN tracks
             JOIN albums
             JOIN artists
             ON customers.customerId = invoices.customerId 
             AND invoices.invoiceId = invoice_items.invoiceId
             AND invoice_items.trackId = tracks.trackId 
             AND tracks.albumId = albums.albumId 
             AND albums.artistId = artists.artistId 
             WHERE artists.name = ?
             GROUP BY customers.country
             ORDER BY SUM(invoice_items.unitPrice) DESC
             LIMIT 5"""
    
    cur.execute(sql, ("Metallica",))
    
    for idx, item in enumerate(cur, start=1):
        print(f"[{idx}]: {item[1]} = {item[0]:.2f} USD")

[1]: USA = 26.73 USD
[2]: Canada = 14.85 USD
[3]: France = 9.90 USD
[4]: United Kingdom = 8.91 USD
[5]: Germany = 8.91 USD


In [48]:
# Cual es el costo de los albums de Metallica
with conn:
    cur = conn.cursor()
    
    sql = """SELECT albums.title, SUM(tracks.unitPrice), COUNT(albums.title)
             FROM tracks
             JOIN albums 
             JOIN artists
             ON tracks.albumId = albums.albumId 
             AND albums.artistId = artists.artistId
             WHERE artists.name = ? 
             GROUP BY albums.albumId
             ORDER BY SUM(tracks.unitPrice)"""
    
    cur.execute(sql, ("Metallica",))
    
    for item in cur:
        print(f"{item[0]:25} \t\t: {item[1]:5.2f} USD \t Tracks: {item[2]:2}")

Master Of Puppets         		:  7.92 USD 	 Tracks:  8
Ride The Lightning        		:  7.92 USD 	 Tracks:  8
...And Justice For All    		:  8.91 USD 	 Tracks:  9
Kill 'Em All              		:  9.90 USD 	 Tracks: 10
Garage Inc. (Disc 1)      		: 10.89 USD 	 Tracks: 11
St. Anger                 		: 10.89 USD 	 Tracks: 11
Black Album               		: 11.88 USD 	 Tracks: 12
ReLoad                    		: 12.87 USD 	 Tracks: 13
Load                      		: 13.86 USD 	 Tracks: 14
Garage Inc. (Disc 2)      		: 15.84 USD 	 Tracks: 16


In [49]:
# Cuales son los artistas mas representativos de la tienda por cada genero musical


In [50]:
# Mostrar los Top 20 de las canciones mas vendidas


In [51]:
# Cuales son los Top 10 canciones mas largas de la tienda (graf. barras)


In [ ]:
# Cuales son los Top 25 clientes de la tienda
